In [1]:

import numpy as np

import torch
import torchvision
import torch.nn as nn
from torch.autograd import Variable


def print_model_param_nums(model=None, multiply_adds=True):
    if model == None:
        model = torchvision.models.alexnet()
    total = sum([param.nelement() for param in model.parameters()])
    print('  + Number of params: %.2fM' % (total / 1e6))

def print_model_param_flops(model=None, input_res=224, multiply_adds=True):

    prods = {}
    def save_hook(name):
        def hook_per(self, input, output):
            prods[name] = np.prod(input[0].shape)
        return hook_per

    list_1=[]
    def simple_hook(self, input, output):
        list_1.append(np.prod(input[0].shape))
    list_2={}
    def simple_hook2(self, input, output):
        list_2['names'] = np.prod(input[0].shape)

    list_conv=[]
    def conv_hook(self, input, output):
        batch_size, input_channels, input_height, input_width = input[0].size()
        output_channels, output_height, output_width = output[0].size()

        kernel_ops = self.kernel_size[0] * self.kernel_size[1] * (self.in_channels / self.groups)
        bias_ops = 1 if self.bias is not None else 0

        params = output_channels * (kernel_ops + bias_ops)
        flops = (kernel_ops * (2 if multiply_adds else 1) + bias_ops) * output_channels * output_height * output_width * batch_size

        list_conv.append(flops)

    list_linear=[]
    def linear_hook(self, input, output):
        batch_size = input[0].size(0) if input[0].dim() == 2 else 1

        weight_ops = self.weight.nelement() * (2 if multiply_adds else 1)
        bias_ops = self.bias.nelement()

        flops = batch_size * (weight_ops + bias_ops)
        list_linear.append(flops)

    list_bn=[]
    def bn_hook(self, input, output):
        list_bn.append(input[0].nelement() * 2)

    list_relu=[]
    def relu_hook(self, input, output):
        list_relu.append(input[0].nelement())

    list_pooling=[]
    def pooling_hook(self, input, output):
        batch_size, input_channels, input_height, input_width = input[0].size()
        output_channels, output_height, output_width = output[0].size()

        kernel_ops = self.kernel_size * self.kernel_size
        bias_ops = 0
        params = 0
        flops = (kernel_ops + bias_ops) * output_channels * output_height * output_width * batch_size

        list_pooling.append(flops)

    list_upsample=[]
    # For bilinear upsample
    def upsample_hook(self, input, output):
        batch_size, input_channels, input_height, input_width = input[0].size()
        output_channels, output_height, output_width = output[0].size()

        flops = output_height * output_width * output_channels * batch_size * 12
        list_upsample.append(flops)

    def foo(net):
        childrens = list(net.children())
        if not childrens:
            if isinstance(net, torch.nn.Conv2d):
                net.register_forward_hook(conv_hook)
            if isinstance(net, torch.nn.Linear):
                net.register_forward_hook(linear_hook)
            if isinstance(net, torch.nn.BatchNorm2d):
                net.register_forward_hook(bn_hook)
            if isinstance(net, torch.nn.ReLU):
                net.register_forward_hook(relu_hook)
            if isinstance(net, torch.nn.MaxPool2d) or isinstance(net, torch.nn.AvgPool2d):
                net.register_forward_hook(pooling_hook)
            if isinstance(net, torch.nn.Upsample):
                net.register_forward_hook(upsample_hook)
            return
        for c in childrens:
            foo(c)

    if model == None:
        model = torchvision.models.alexnet()
    foo(model)
    input = Variable(torch.rand(3, 3, input_res, input_res), requires_grad = True)
    out = model(input)


    total_flops = (sum(list_conv) + sum(list_linear) + sum(list_bn) + sum(list_relu) + sum(list_pooling) + sum(list_upsample))

    print('  + Number of FLOPs: %.5fG' % (total_flops / 3 / 1e9))
    
    return total_flops / 3


In [2]:
import argparse
import numpy as np
import os
import shutil

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [3]:
import math

import torch
import torch.nn as nn
import torch.nn.functional as F
from functools import partial
from torch.autograd import Variable


__all__ = ['resnet']

def conv3x3(in_planes, out_planes, stride=1):
    "3x3 convolution with padding"
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, cfg, stride=1, downsample=None):
        # cfg should be a number in this case
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, cfg, stride)
        self.bn1 = nn.BatchNorm2d(cfg)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(cfg, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out

def downsample_basic_block(x, planes):
    x = nn.AvgPool2d(2,2)(x)
    zero_pads = torch.Tensor(
        x.size(0), planes - x.size(1), x.size(2), x.size(3)).zero_()
    if isinstance(x.data, torch.cuda.FloatTensor):
        zero_pads = zero_pads.cuda()

    out = Variable(torch.cat([x.data, zero_pads], dim=1))

    return out

class ResNet(nn.Module):

    def __init__(self, depth, dataset='cifar10', cfg=None):
        super(ResNet, self).__init__()
        # Model type specifies number of layers for CIFAR-10 model
        assert (depth - 2) % 6 == 0, 'depth should be 6n+2'
        n = (depth - 2) // 6

        block = BasicBlock
        if cfg == None:
            cfg = [[16]*n, [32]*n, [64]*n]
            cfg = [item for sub_list in cfg for item in sub_list]

        self.cfg = cfg

        self.inplanes = 16
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self._make_layer(block, 16, n, cfg=cfg[0:n])
        self.layer2 = self._make_layer(block, 32, n, cfg=cfg[n:2*n], stride=2)
        self.layer3 = self._make_layer(block, 64, n, cfg=cfg[2*n:3*n], stride=2)
        self.avgpool = nn.AvgPool2d(8)
        if dataset == 'cifar10':
            num_classes = 10
        elif dataset == 'cifar100':
            num_classes = 100
        self.fc = nn.Linear(64 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, cfg, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = partial(downsample_basic_block, planes=planes*block.expansion)

        layers = []
        layers.append(block(self.inplanes, planes, cfg[0], stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes, cfg[i]))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)    # 32x32

        x = self.layer1(x)  # 32x32
        x = self.layer2(x)  # 16x16
        x = self.layer3(x)  # 8x8

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

def resnet(**kwargs):
    """
    Constructs a ResNet model.
    """
    return ResNet(**kwargs)

if __name__ == '__main__':
    net = resnet(depth=50)

In [6]:
dataset = 'cifar100'
depth = 50
batch_size = 64
test_batch_size = 256
epochs = 50
lr = 0.001
log_interval = 100
save = '/kaggle/working/'
arch = 'resnet'
seed = 1
momentum = 0.9
weight_decay = 1e-4

In [7]:
train_loader = torch.utils.data.DataLoader(
        datasets.CIFAR100('./data.cifar100', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.Pad(4),
                           transforms.RandomCrop(32),
                           transforms.RandomHorizontalFlip(),
                           transforms.ToTensor(),
                           transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
                       ])),
        batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(
        datasets.CIFAR100('./data.cifar100', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
                       ])),
        batch_size=test_batch_size, shuffle=True)

Files already downloaded and verified


In [8]:
model = resnet(dataset=dataset, depth=depth)

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if not os.path.exists(save):
    os.makedirs(save)

In [10]:
model.to(device)

ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=Fals

In [12]:
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)

In [13]:
def train(epoch):
    model.train()
    avg_loss = 0.
    train_acc = 0.
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        
        optimizer.zero_grad()
        
        # Forward pass: Compute model output
        output = model(data)
        
        # Compute loss
        loss = F.cross_entropy(output, target)
        avg_loss = avg_loss + loss.item()
        
        # Use torch.no_grad() for accuracy computation (no gradients required here)
        with torch.no_grad():
            pred = output.data.max(1, keepdim=True)[1]  # Get index of max log-probability
            train_acc += pred.eq(target.data.view_as(pred)).cpu().sum()

        # Backpropagate the loss and optimize the model
        loss.backward()
        optimizer.step()
        
        # Log progress
        if batch_idx % log_interval == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} '
                  f'({100. * batch_idx / len(train_loader):.1f}%)]\tLoss: {loss.item():.6f}')


In [18]:
from torch.profiler import profile, ProfilerActivity

def test():
    model.eval()
    test_loss = 0
    correct = 0

    # Start profiling
    with profile(
        activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA],
        record_shapes=True,
        profile_memory=True,
        with_stack=True,
    ) as prof:  # Profiler object initialized here
        with torch.no_grad():  # Use torch.no_grad() for inference
            for data, target in test_loader:
                data, target = data.cuda(), target.cuda()
                output = model(data)
                test_loss += F.cross_entropy(output, target, reduction='sum').item()  # sum batch loss
                pred = output.argmax(dim=1, keepdim=True)  # Get the index of the max log-probability
                correct += pred.eq(target.view_as(pred)).sum().item()

    prof.export_chrome_trace("trace.json")

    # Calculate test loss and accuracy
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.1f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

    return correct / len(test_loader.dataset)

In [20]:
def save_checkpoint(state, is_best, filepath):
    torch.save(state, os.path.join(filepath, 'checkpoint.pth.tar'))
    if is_best:
        shutil.copyfile(os.path.join(filepath, 'checkpoint.pth.tar'), os.path.join(filepath, 'model_best.pth.tar'))

In [23]:
best_prec1 = 0
epochs = 30
for epoch in range(0, epochs):
    if epoch in [epochs*0.5, epochs*0.75]:
        for param_group in optimizer.param_groups:
            param_group['lr'] *= 0.001
    train(epoch)
    prec1 = test()
    is_best = prec1 > best_prec1
    best_prec1 = max(prec1, best_prec1)
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'best_prec1': best_prec1,
        'optimizer': optimizer.state_dict(),
        'cfg': model.cfg
    }, is_best, filepath=save)

Train Epoch: 0 [0/50000 (0.0%)]	Loss: 2.543618
Train Epoch: 0 [6400/50000 (12.8%)]	Loss: 2.356167
Train Epoch: 0 [12800/50000 (25.6%)]	Loss: 2.022959
Train Epoch: 0 [19200/50000 (38.4%)]	Loss: 2.233490
Train Epoch: 0 [25600/50000 (51.2%)]	Loss: 1.847636
Train Epoch: 0 [32000/50000 (63.9%)]	Loss: 2.182014
Train Epoch: 0 [38400/50000 (76.7%)]	Loss: 2.446960
Train Epoch: 0 [44800/50000 (89.5%)]	Loss: 2.261799

Test set: Average loss: 2.3250, Accuracy: 3917/10000 (39.2%)

Train Epoch: 1 [0/50000 (0.0%)]	Loss: 2.235600
Train Epoch: 1 [6400/50000 (12.8%)]	Loss: 2.391966
Train Epoch: 1 [12800/50000 (25.6%)]	Loss: 2.266216
Train Epoch: 1 [19200/50000 (38.4%)]	Loss: 2.073068
Train Epoch: 1 [25600/50000 (51.2%)]	Loss: 2.209569
Train Epoch: 1 [32000/50000 (63.9%)]	Loss: 2.207880
Train Epoch: 1 [38400/50000 (76.7%)]	Loss: 2.016623
Train Epoch: 1 [44800/50000 (89.5%)]	Loss: 2.083930

Test set: Average loss: 2.3147, Accuracy: 3956/10000 (39.6%)

Train Epoch: 2 [0/50000 (0.0%)]	Loss: 2.392545
Train E

In [24]:
if save:
    save_path = os.path.join(save, 'resnet_model.pth')
    torch.save(model.state_dict(), save_path)
    print(f"Model saved to {save_path}")

Model saved to /kaggle/working/resnet_model.pth


# Accuracy and Number of Parameters for the Baseline Model

In [ ]:
base_model = torch.load('resnet_model.pth')

In [ ]:
print('Accuracy for the Baseline model')
acc = test()

In [25]:
num_parameters = sum([param.nelement() for param in model.parameters()])
print(num_parameters)

761652


In [38]:
import torch.nn.utils.prune as prune
import torch.quantization as quant
from torch.profiler import profile, record_function, ProfilerActivity
import time

def profile_model(model, data_loader, num_batches=10):
    model.eval()
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.to(device)

    with profile(
        activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA],
        on_trace_ready=torch.profiler.tensorboard_trace_handler('./logs'),
        record_shapes=True,
        with_stack=True
    ) as prof:
        for batch_idx, (inputs, targets) in enumerate(data_loader):
            if batch_idx >= num_batches:
                break
            inputs, targets = inputs.to(device), targets.to(device)

            start_time = time.time()
            with record_function("model_inference"):
                outputs = model(inputs)
            latency = time.time() - start_time

    print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=10))
    print(f"\nLatency per batch: {latency:.4f} seconds")
    print(f"Throughput: {len(inputs) / latency:.4f} samples/second")
    print(f"Peak GPU memory usage: {torch.cuda.max_memory_allocated(device) / (1024 * 1024):.2f} MB")
    total_params = sum(p.numel() for p in model.parameters())
    print(f"Model parameter count: {total_params}")
    print(f"FLOPs estimation: {total_params * 2:.2e} FLOPs")



In [41]:
profile_model(model, test_loader)

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                        model_inference         0.00%       0.000us         0.00%       0.000us       0.000us     454.575ms        50.58%     454.575ms      45.457ms            10  
                                        model_inference         4.06%      71.153ms        25.74%     451.576ms      45.158ms       0.000us         0.00%     438.587ms      43.859ms            10  
         

# Pruning

In [64]:
skip = {
    'A': [16, 20, 38, 54],
    'B': [16, 18, 20, 34, 38, 54],
}

prune_prob = {
    'A': [0.1, 0.1, 0.1],
    'B': [0.6, 0.3, 0.1],
}

layer_id = 1
cfg = []
cfg_mask = []

In [65]:
for m in model.modules():
    if isinstance(m, nn.Conv2d):
        out_channels = m.weight.data.shape[0]
        if layer_id in skip['A']:
            cfg_mask.append(torch.ones(out_channels))
            cfg.append(out_channels)
            layer_id += 1
            continue
        if layer_id % 2 == 0:
            if layer_id <= 18:
                stage = 0
            elif layer_id <= 36:
                stage = 1
            else:
                stage = 2
            prune_prob_stage = prune_prob['A'][stage]
            weight_copy = m.weight.data.abs().clone().cpu().numpy()
            L1_norm = np.sum(weight_copy, axis=(1,2,3))
            num_keep = int(out_channels * (1 - prune_prob_stage))
            arg_max = np.argsort(L1_norm)
            arg_max_rev = arg_max[::-1][:num_keep]
            mask = torch.zeros(out_channels)
            mask[arg_max_rev.tolist()] = 1
            cfg_mask.append(mask)
            cfg.append(num_keep)
            layer_id += 1
            continue
        layer_id += 1

In [66]:
newmodel = model

In [67]:
start_mask = torch.ones(3)
layer_id_in_cfg = 0
conv_count = 1
for [m0, m1] in zip(model.modules(), newmodel.modules()):
    if isinstance(m0, nn.Conv2d):
        if conv_count == 1:
            m1.weight.data = m0.weight.data.clone()
            conv_count += 1
            continue
        if conv_count % 2 == 0:
            mask = cfg_mask[layer_id_in_cfg]
            idx = np.squeeze(np.argwhere(np.asarray(mask.cpu().numpy())))
            if idx.size == 1:
                idx = np.resize(idx, (1,))
            w = m0.weight.data[idx.tolist(), :, :, :].clone()
            m1.weight.data = w.clone()
            layer_id_in_cfg += 1
            conv_count += 1
            continue
        if conv_count % 2 == 1:
            mask = cfg_mask[layer_id_in_cfg-1]
            idx = np.squeeze(np.argwhere(np.asarray(mask.cpu().numpy())))
            if idx.size == 1:
                idx = np.resize(idx, (1,))
            w = m0.weight.data[:, idx.tolist(), :, :].clone()
            m1.weight.data = w.clone()
            conv_count += 1
            continue
    elif isinstance(m0, nn.BatchNorm2d):
        if conv_count % 2 == 1:
            mask = cfg_mask[layer_id_in_cfg-1]
            idx = np.squeeze(np.argwhere(np.asarray(mask.cpu().numpy())))
            if idx.size == 1:
                idx = np.resize(idx, (1,))
            m1.weight.data = m0.weight.data[idx.tolist()].clone()
            m1.bias.data = m0.bias.data[idx.tolist()].clone()
            m1.running_mean = m0.running_mean[idx.tolist()].clone()
            m1.running_var = m0.running_var[idx.tolist()].clone()
            continue
        m1.weight.data = m0.weight.data.clone()
        m1.bias.data = m0.bias.data.clone()
        m1.running_mean = m0.running_mean.clone()
        m1.running_var = m0.running_var.clone()
    elif isinstance(m0, nn.Linear):
        m1.weight.data = m0.weight.data.clone()
        m1.bias.data = m0.bias.data.clone()

torch.save({'cfg': cfg, 'state_dict': newmodel.state_dict()}, os.path.join(save, 'pruned.pth.tar'))


In [69]:
num_parameters = sum([param.nelement() for param in newmodel.parameters()])
print(newmodel)



ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=Fals

# Accuracy and Number of Parameters for the Pruned Model

In [70]:
print("number of parameters: "+str(num_parameters))

number of parameters: 562442


In [71]:
acc = test()



Test set: Average loss: 8.9573, Accuracy: 297/10000 (3.0%)



# Finetuning after pruning

In [73]:
optimizer = optim.SGD(newmodel.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)

In [74]:
def train(epoch):
    newmodel.train()
    avg_loss = 0.
    train_acc = 0.
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        
        optimizer.zero_grad()
        
        # Forward pass: Compute model output
        output = newmodel(data)
        
        # Compute loss
        loss = F.cross_entropy(output, target)
        avg_loss = avg_loss + loss.item()
        
        # Use torch.no_grad() for accuracy computation (no gradients required here)
        with torch.no_grad():
            pred = output.data.max(1, keepdim=True)[1]  # Get index of max log-probability
            train_acc += pred.eq(target.data.view_as(pred)).cpu().sum()

        # Backpropagate the loss and optimize the model
        loss.backward()
        optimizer.step()
        
        # Log progress
        if batch_idx % log_interval == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} '
                  f'({100. * batch_idx / len(train_loader):.1f}%)]\tLoss: {loss.item():.6f}')


In [75]:
best_prec1 = 0.
epochs = 50
newmodel.train()
newmodel.to(device)
for epoch in range(0, epochs):
    train(epoch)
    prec1 = test()
    is_best = prec1 > best_prec1
    best_prec1 = max(prec1, best_prec1)
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict': newmodel.state_dict(),
        'best_prec1': best_prec1,
        'optimizer': optimizer.state_dict(),
        'cfg': newmodel.cfg
    }, is_best, filepath=save)
    

Train Epoch: 0 [0/50000 (0.0%)]	Loss: 2.762380
Train Epoch: 0 [6400/50000 (12.8%)]	Loss: 3.021941
Train Epoch: 0 [12800/50000 (25.6%)]	Loss: 2.303818
Train Epoch: 0 [19200/50000 (38.4%)]	Loss: 2.404951
Train Epoch: 0 [25600/50000 (51.2%)]	Loss: 2.334910
Train Epoch: 0 [32000/50000 (63.9%)]	Loss: 2.590052
Train Epoch: 0 [38400/50000 (76.7%)]	Loss: 2.442172
Train Epoch: 0 [44800/50000 (89.5%)]	Loss: 2.385309

Test set: Average loss: 2.4715, Accuracy: 3590/10000 (35.9%)

Train Epoch: 1 [0/50000 (0.0%)]	Loss: 2.401487
Train Epoch: 1 [6400/50000 (12.8%)]	Loss: 2.424259
Train Epoch: 1 [12800/50000 (25.6%)]	Loss: 2.481336
Train Epoch: 1 [19200/50000 (38.4%)]	Loss: 2.878996
Train Epoch: 1 [25600/50000 (51.2%)]	Loss: 2.409033
Train Epoch: 1 [32000/50000 (63.9%)]	Loss: 2.456629
Train Epoch: 1 [38400/50000 (76.7%)]	Loss: 2.498177
Train Epoch: 1 [44800/50000 (89.5%)]	Loss: 2.518283

Test set: Average loss: 2.3801, Accuracy: 3848/10000 (38.5%)

Train Epoch: 2 [0/50000 (0.0%)]	Loss: 2.602745
Train E

# Accuracy and Number of Parameters for the Finetuned Pruned Model

In [79]:
newmodel.eval()
print('Accuracy for the Baseline model')
acc = test()

Accuracy for the Baseline model

Test set: Average loss: 1.7871, Accuracy: 5236/10000 (52.4%)



In [80]:
num_parameters = sum([param.nelement() for param in newmodel.parameters()])
print(num_parameters)

562442


In [81]:
profile_model(newmodel, test_loader)

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                        model_inference         0.00%       0.000us         0.00%       0.000us       0.000us     424.108ms        51.28%     424.108ms      21.205ms            20  
                                        model_inference         3.09%      53.850ms        22.89%     399.329ms      39.933ms       0.000us         0.00%     397.623ms      39.762ms            10  
         